In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px 
from plotly.offline import iplot

from sklearn.model_selection import train_test_split, KFold, cross_val_score, RandomizedSearchCV
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from imblearn.combine import SMOTETomek

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer


import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping

import warnings


# set Default
pd.set_option("display.max_columns", None)
pd.options.display.float_format = '{:,.2f}'.format
warnings.filterwarnings("ignore")
pd.options.mode.copy_on_write = "warn"

# Used Color
used_color = ["#ED1C24", "#FAF7F1", "#C0C0C0", "#330072", "#F1ECDF"]

In [ ]:
def update_layout(
    title_font_size = 28,
    hover_font_size = 16,
    hover_bgcolor = '#111',
    showlegend = False
):
    fig.update_layout(
        showlegend = showlegend,
        title = {
            'font' : {
                'size' : title_font_size,
                'family' : '<b>poppins'
            }
        },
        
        hoverlabel = {
            'bgcolor' : hover_bgcolor,
            'font_size' : hover_font_size,
            'font_family' : 'poppins'
        }
    )

In [ ]:
def count_viz_func(
    data_frame,
    column_name,
    title = 'Chart Title',
    title_font_size = 30,
    x_label = 'X',
    y_label = 'Y',
    showlegend = False,
    hover_template = 'None'
):
    value_counts = df[column_name].value_counts(normalize=1)*100
    
    fig = px.bar(
        data_frame = value_counts,
        x = value_counts.index,
        y = value_counts,
        color = value_counts.index,
        color_discrete_sequence = used_color,
        title = title,
        labels = {column_name:x_label, 'y': y_label},
        template = 'plotly_dark',
        text = value_counts.apply(lambda x: f'{x:.1f}%')
    )
    
    fig.update_layout(
        showlegend = showlegend,
        title = {
            'font' : {
                'size' : title_font_size,
                'family' : '<b>poppins'
            }
        },
        
        hoverlabel = {
            'bgcolor' : '#222',
            'font_size' : 16,
            'font_family' : 'poppins'
        }
    )
    
    fig.update_traces(
        textfont = {
            'size' : 18,
            'family' : 'consolas',
            'color' : '#222'
        },
        
        marker = dict(line=dict(color = '#222', width=1)),
        
        hovertemplate = hover_template
    )
    
    return fig

### __About Dataset__
* `Senior Citizen`: Indicates if the customer is 65 or older: Yes, No
* `Dependents`: Indicates if the customer lives with any dependents: Yes, No. Dependents could be children, parents, grandparents, etc
* `Tenure in Months`: Indicates the total amount of months that the customer has been with the company
* `Internet Service`: Indicates if the customer subscribes to Internet service with the company: No, DSL, Fiber Optic, Cable.
* `Contract`: Indicates the customer’s current contract type: Month-to-Month, One Year, Two Year.
* `Payment Method`: Indicates how the customer pays their bill: Bank Withdrawal, Credit Card, Mailed Check.
* `Monthly Charge`: Indicates the customer’s current total monthly charge for all their services from the company.
* `Total Charges`: Indicates the customer’s total charges, calculated to the end of the quarter specified above.
* `Churn: 1` = the customer left the company this quarter. 0 = the customer remained with the company

In [ ]:
df = pd.read_csv('./WA_Fn-UseC_-Telco-Customer-Churn.csv')

In [ ]:
print(f'Number of Records: {df.shape[0]:.0f}')
print(f'Number of Features: {df.shape[1]}')

In [ ]:
df.head()

### __Data Cleaning & Wrangling__

In [ ]:
# Check For Null Values
df.isna().sum()

In [ ]:
# Duplicated Records
df.duplicated().sum()

In [ ]:
# Check for Duplicated Customer ID
df['customerID'].duplicated().sum()

__Data almost clean and it's important that the `customerID` is unique for each customer because of it we need to remove the `customerID`__

In [ ]:
df.drop(columns='customerID', inplace=True)

In [ ]:
df.describe().T

### __Dig Deeper Into Each Column__

##### __Gender Column__

In [ ]:
gender = df["gender"].value_counts(normalize=1)*100
gender

In [ ]:
fig = count_viz_func(
    df, 'gender', title= '<b>Gender Distributions', x_label='<b>Gender', y_label='<b>Frequency in PCT (%)', hover_template='Gender: %{x}<br>Frequency PCT (%): %{y:.0f}'
)

iplot(fig)

In [ ]:
len(df[df['Churn'] == 'Yes']) / len(df) * 100

##### __Senior Citizen Column__

In [ ]:
f = df['Contract'] == 'Month-to-month'
df.loc[f, 'PaymentMethod']

In [ ]:
senior_citizen = df['SeniorCitizen'].value_counts(normalize=1)*100
senior_citizen.apply(lambda x: f'{x:.2f}%')

In [ ]:
fig = px.bar(
    data_frame= senior_citizen,
    x= ['Greater Than 65' if i == 1 else 'Less Than 65' for i in senior_citizen.index],
    y= senior_citizen,
    color= ['Greater Than 65' if i == 1 else 'Less Than 65' for i in senior_citizen.index],
    color_discrete_sequence=used_color,
    title = '<b>Senior Citizen Distributions',
    labels = {'x': '<b>Is Senior Citizen', 'y': '<b>Frequency in PCT (%)'},
    template = 'plotly_dark',
    text = senior_citizen.apply(lambda x: f'{x:.1f}%')
)

update_layout()

fig.update_traces(
    textfont = {
        'size': 18,
        'family' : 'consolas',
        'color' : '#222'
    },
    
    marker = dict(line = dict(color = '#C0C0C0', width=2)),
    hovertemplate = 'Category: %{x}<br>Frequency in PCT (%): %{y:.0f}'
)
iplot(fig)

##### __Dependents Column__

In [ ]:
dependents = (df['Dependents'].value_counts(normalize=1)*100).apply(lambda x: f'{x:.2f}%')
dependents

In [ ]:
fig = count_viz_func(
    df, 'Dependents',
    title='<b>Dependents Distributions',
    x_label = '<b>Dependents',
    y_label = '<b>Frequency in PCT (%)',
    hover_template = 'Dependents: %{x}<br>Frequency in PCT (%): %{y:.0f}'
)

iplot(fig)

##### __Tenure Column__

In [ ]:
fig = px.box(
    x=df['tenure'],
    template='plotly_dark',
    labels={'x': 'Tenure in Months'},
    title='<b>Tenure in Months',
    notched=True
)

fig.update_traces(
    marker = dict(color='red'),
    line = dict(color='red')
)

update_layout()
iplot(fig)

In [ ]:
print(f'Tenure Skewness: {df["tenure"].skew():.4f}')

### Insights based on Boxplot Statistic Tenure

**Insight Boxplot:**

1. **Variasi dalam Tenure:**
   - Tenure pelanggan bervariasi dari 0 hingga 72 bulan. Ini menunjukkan bahwa ada pelanggan yang baru saja bergabung (0 bulan) dan ada yang sudah lama (hingga 72 bulan).

2. **Distribusi Pelanggan:**
   - **Median (29 bulan):** Separuh dari pelanggan memiliki masa berlangganan kurang dari 29 bulan dan separuh lainnya lebih dari 29 bulan.
   - **Kuartil Pertama (Q1 - 9 bulan):** 25% dari pelanggan memiliki masa berlangganan 9 bulan atau kurang, menunjukkan bahwa sebagian besar pelanggan baru memiliki masa berlangganan pendek.
   - **Kuartil Ketiga (Q3 - 55 bulan):** 75% dari pelanggan memiliki masa berlangganan hingga 55 bulan atau kurang. Hanya 25% pelanggan yang memiliki masa berlangganan lebih dari itu.

3. **Pelanggan Baru vs. Pelanggan Lama:**
   - Adanya nilai minimum dan lower fence pada 0 bulan menunjukkan bahwa ada beberapa pelanggan yang sangat baru.
   - Nilai maksimum dan upper fence di 72 bulan menunjukkan bahwa beberapa pelanggan sangat setia dan telah bersama perusahaan untuk waktu yang lama.

4. **Keberagaman Pelanggan:**
   - Distribusi ini menunjukkan adanya keberagaman yang signifikan dalam masa berlangganan pelanggan, yang dapat mengindikasikan berbagai tingkat kepuasan dan loyalitas pelanggan.

### Insights dari Skewness Tenure

**Output Skewness:**
- **Tenure Skewness:** 0.2395

**Insight:**

1. **Distribusi Tenure yang Mendekati Simetris:**
   - Skewness (kemiringan) sebesar 0.2395 menunjukkan bahwa distribusi masa berlangganan (tenure) pelanggan hampir simetris, tetapi sedikit condong ke kanan.

2. **Impilikasi untuk Strategi Retensi:**
   - Skewness yang rendah menunjukkan bahwa sebagian besar pelanggan memiliki masa berlangganan yang tersebar cukup merata di sekitar median (29 bulan). Hal ini berarti tidak ada dominasi pelanggan baru atau pelanggan lama dalam data.
   - Sedikit condong ke kanan menunjukkan bahwa ada sedikit lebih banyak pelanggan dengan masa berlangganan yang lebih lama daripada yang baru, tetapi perbedaannya tidak signifikan.

3. **Kesempatan untuk Meningkatkan Loyalitas:**
   - Dengan distribusi yang mendekati simetris ini, perusahaan dapat mengembangkan strategi retensi yang lebih seimbang, berfokus pada menjaga pelanggan yang ada dan meningkatkan masa berlangganan pelanggan baru.

4. **Evaluasi Program Loyalitas:**
   - Melihat distribusi yang hampir simetris ini, program loyalitas bisa dievaluasi untuk melihat bagaimana program ini mempengaruhi pelanggan dengan berbagai lama berlangganan. Upaya bisa difokuskan untuk meningkatkan masa berlangganan pelanggan yang berada di bawah median.

In [ ]:
df['tenure'].describe()